In [1]:
import urllib
import os
import numpy as np
import pynwb
import pandas as pd

from ndx_icephys_meta import *

#import NWBWidgets. For details about nwbwidgets see https://github.com/NeurodataWithoutBorders/nwb-jupyter-widgets
from nwbwidgets import nwb2widget
from nwbwidgets.timeseries import show_indexed_timeseries_mpl

#import matplotlib
#from matplotlib import pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')

# # Download an example Intracellular EPhys file from DANDI
# The file we use here is from the Allen Institute for Brain Science as part of 
# https://portal.brain-map.org/explore/classes/multimodal-characterization/multimodal-characterization-mouse-visual-cortex . 
# The data is available on DANDI as part of https://gui.dandiarchive.org/#/dandiset/000020. 
# We here use a single NWB file from this collection as an example.

#url = 'https://girder.dandiarchive.org/api/v1/item/5f00fd6ba78c4b52bbb38e81/download'
#filename = 'sub-1001658946_ses-1003020741_icephys.nwb'

#if not os.path.exists(filename):
#    print("Downloading file...")
#    res = urllib.request.urlretrieve(url, filename)

filename = '/Users/pam.baker/Downloads/001_140709EXP_A1_ndx_v3.nwb'
    
reader = pynwb.NWBHDF5IO(filename, mode='r', load_namespaces=True)
nwbfile = reader.read()

/opt/anaconda3/envs/pynwb_widgets/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/pynwb_widgets/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'core' version 2.1.0 because version 2.3.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [144]:
#yay = nwbfile.intracellular_recordings.to_dataframe()
#display(yay['stimulus_type'][:])

dir(nwbfile)

['_AbstractContainer__children',
 '_AbstractContainer__container_source',
 '_AbstractContainer__field_values',
 '_AbstractContainer__fieldsconf',
 '_AbstractContainer__gather_fields',
 '_AbstractContainer__modified',
 '_AbstractContainer__name',
 '_AbstractContainer__object_id',
 '_AbstractContainer__parent',
 '_Container__smart_str',
 '_Container__smart_str_dict',
 '_Container__smart_str_list',
 '_MultiContainerInterface__add_article',
 '_MultiContainerInterface__build_class',
 '_MultiContainerInterface__build_conf_methods',
 '_MultiContainerInterface__join',
 '_MultiContainerInterface__make_add',
 '_MultiContainerInterface__make_constructor',
 '_MultiContainerInterface__make_create',
 '_MultiContainerInterface__make_get',
 '_MultiContainerInterface__make_getitem',
 '_MultiContainerInterface__make_getter',
 '_MultiContainerInterface__make_setter',
 '_NWBFile__check_electrodes',
 '_NWBFile__check_epochs',
 '_NWBFile__check_invalid_times',
 '_NWBFile__check_trials',
 '_NWBFile__check_un

In [181]:
yay = nwbfile.ic_sweep_sequences.to_dataframe()

# also a dataframe
yay_df = yay['sweeps'].values[15]

# row entry from intracellular_recordings
yay_df['recordings'].values[0]

,stimulus,response,electrode,stimulus_type
id,,,,
33,"[[0, 35000, ccss_IV_0044 pynwb.icephys.Current...","[[0, 35000, ccs_IV_0044 pynwb.icephys.CurrentC...",[Electrode_01 pynwb.icephys.IntracellularElect...,[IV]


## Getting Data and Stimuli and writing to new file

In [2]:
iceall_df = nwbfile.intracellular_recordings.to_dataframe()

In [3]:
unique_stimulus_types = np.unique(iceall_df['stimulus_type'])
print(unique_stimulus_types)

['APWaveform' 'DeHyperPol' 'Delta' 'ElecCal' 'FirePattern' 'HyperDePol'
 'IDRest' 'IDThres' 'IV' 'NegCheops' 'PosCheops' 'RPip' 'RSealClose'
 'RSealOpen' 'Rac' 'ResetITC' 'SetAmpl' 'SetISI' 'SineSpec' 'SponHold3'
 'SponHold30' 'SponNoHold30' 'StartHold' 'StartNoHold' 'TestAmpl'
 'TestRheo' 'sAHP']


In [186]:
#iceIV_df = iceall_df[iceall_df['stimulus_type']=='IV']
#iceIV_df

iceSineSpec_df = iceall_df[iceall_df['stimulus_type']=='SineSpec']
iceSineSpec_df

,stimulus,response,electrode,stimulus_type
id,,,,
129,"(0, 52000, ccss_SineSpec_0140 pynwb.icephys.Cu...","(0, 52000, ccs_SineSpec_0140 pynwb.icephys.Cur...",Electrode_01 pynwb.icephys.IntracellularElectr...,SineSpec
130,"(0, 52000, ccss_SineSpec_0141 pynwb.icephys.Cu...","(0, 52000, ccs_SineSpec_0141 pynwb.icephys.Cur...",Electrode_01 pynwb.icephys.IntracellularElectr...,SineSpec
131,"(0, 52000, ccss_SineSpec_0142 pynwb.icephys.Cu...","(0, 52000, ccs_SineSpec_0142 pynwb.icephys.Cur...",Electrode_01 pynwb.icephys.IntracellularElectr...,SineSpec


In [194]:
# get stimulus and response columns only as arrays
stimsall = nwbfile.intracellular_recordings['stimulus'].data[:]
respsall = nwbfile.intracellular_recordings['response'].data[:]

# get traces for first IV curve stimulus runs stored as numpy array
fst = 128
lst = 131
stims = np.concatenate(([stimsall[i][2].data[()].reshape(-1,1) for i in range(fst,lst,1)]), axis=1)
resps = np.concatenate(([respsall[i][2].data[()].reshape(-1,1) for i in range(fst,lst,1)]), axis=1)

# Collecting start times and stim descriptions here too, tho redundant
start_times = np.array([stimsall[i][2].starting_time for i in range(fst,lst,1)])
stim_desc = np.array([stimsall[i][2].stimulus_description for i in range(fst,lst,1)])
rates = np.array([stimsall[i][2].rate for i in range(fst,lst,1)])
sweep_nums = np.array([i for i in range(fst,lst,1)])

In [134]:
stims[0][0:20]

array([-7.0750e-11, -7.0750e-11, -7.1375e-11, -7.0625e-11, -7.0750e-11,
       -7.1000e-11, -7.0875e-11, -7.0625e-11, -7.0875e-11, -7.0625e-11,
       -7.1000e-11], dtype=float32)

In [195]:
# Saving arrays to file

np.savez('SineSpec_001_140709EXP_A1_ndx_v3',
         stims=stims,
         resps=resps,
         starts=start_times,
         rates=rates,
         sweep_nums=sweep_nums,
         stim_names=stim_desc)


In [198]:
# ccss is individual trace as hdf5 dataset
# ccss.data[()] is a numpy array with just the values


# can take last 4 chars of the timeseries.name for their defined sweep number
ccss = stimsall[120][2]
ccss.name

'ccss_IDRest_0132'

In [114]:
# alternate access to traces just using dataframe f'ns
stim = iceIV_df.loc[42, ['stimulus']]
stimarr = stim.array.to_numpy()
stimarr[0][2].data[()].shape

(35000,)

In [200]:
nwbfile.fields

{'session_description': 'ASS',
 'identifier': '001_140709RP_A1',
 'session_start_time': datetime.datetime(2014, 7, 9, 14, 16, 36, tzinfo=tzutc()),
 'timestamps_reference_time': datetime.datetime(2014, 7, 9, 14, 16, 36, tzinfo=tzutc()),
 'file_create_date': [datetime.datetime(2020, 2, 4, 17, 7, 26, 773045, tzinfo=tzoffset(None, -28800))],
 'acquisition': {'ccs_APWaveform_0055': ccs_APWaveform_0055 pynwb.icephys.CurrentClampSeries at 0x5285419280
  Fields:
    comments: 
  VERSION#2
  DATE#09/07/2014 (4)
  TIME#14:22:55
  DATETIME#3487760575
  PROTOCOLDEF##	Stimulus	Name	In	Out	Sweep	Rep	Wait	Group	Execute	Trig	
  PROTOCOL#18	14	APWaveform	4;5;	2;	6	1	1.5				
  STIMULUSDEF##	Name	Analog	Parameters	Digital	Units	Scale	Sampling	Duration	
  STIMULUS0#14	APWaveform	Pulse($,0.25,0.45)	60:40:260		A	1e-12	20	0.95	
  SWEEPNUM#0/6
  GROUP#
  EXECUTE#
  TRIG#
  
  PATTERN#14
  IN#4;5;
  OUT#2;
  SWEEP#6
  REP#1
  WAIT#1.5
  PATTERNNAME0#APWaveform
  ANALOG0#Pulse($,0.25,0.45)
  SWEEP0#60:40:260
  

In [185]:
nwbfile.session_description

'ASS'